<a href="https://colab.research.google.com/github/AYOUBSEDDOUKI/AYOUBSEDDOUKI/blob/main/NLP%26TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%time
import os
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import numpy as np
import pandas as pd
import sklearn

# Libraries and packages for text (pre-)processing 
import string
import re
import nltk

print("Python version:", sys.version)
print("Version info.:", sys.version_info)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)
print("skearn version:", sklearn.__version__)
print("re version:", re.__version__)
print("nltk version:", nltk.__version__)

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs
Python version: 3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]
Version info.: sys.version_info(major=3, minor=7, micro=13, releaselevel='final', serial=0)
pandas version: 1.3.5
numpy version: 1.21.5
skearn version: 1.0.2
re version: 2.2.1
nltk version: 3.2.5


In [ ]:
%time

# read the csv file
train_df = pd.read_csv("/content/SPAM text message 20170820 - Data.csv")
display(train_df.shape, train_df.head())

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


(5572, 2)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
display(train_df[~train_df["Message"].isnull()].head())
display(train_df[train_df["Category"] == "ham"]["Message"].values[1])
display(train_df[train_df["Category"] == "spam"]["Message"].values[1])

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


'Ok lar... Joking wif u oni...'

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"

In [ ]:
# tronsformer les message en miniscule
train_df["message_clean"] = train_df["Message"].apply(lambda x: x.lower())
display(train_df.head())

,Category,Message,message_clean
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro..."


In [ ]:
!pip install contractions

     |████████████████████████████████| 106 kB 11.5 MB/s 
     |████████████████████████████████| 287 kB 49.0 MB/s 


In [ ]:
%time
import contractions

# Test
test_text = """
            Y'all can't expand contractions I'd think. I'd like to know how I'd done that! 
            We're going to the zoo and I don't think I'll be home for dinner.
            Theyre going to the zoo and she'll be home for dinner.
            We should've do it in here but we shouldn't've eat it
            """
print("Test: ", contractions.fix(test_text))

# eleminer les "short form"
train_df["message_clean"] = train_df["message_clean"].apply(lambda x: contractions.fix(x))
display(train_df.head())


In [ ]:
# supprimer les liens hypertext
def remove_URL(text):
    """
        Remove URLs from a sample string
    """
    return re.sub(r"https?://\S+|www\.\S+", "", text)

In [ ]:
train_df["message_clean"] = train_df["message_clean"].apply(lambda x: remove_URL(x))
display(train_df.head())

,Category,Message,message_clean
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif you oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,you dun say so early hor... you c already then...
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i do not think he goes to usf, he lives ar..."


In [ ]:
def remove_html(text):
    """
        Remove the html in sample text
    """
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)

In [ ]:
# supprimer les symbols html 
train_df["message_clean"] = train_df["message_clean"].apply(lambda x: remove_html(x))
display(train_df.head())

,Category,Message,message_clean
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif you oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,you dun say so early hor... you c already then...
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i do not think he goes to usf, he lives ar..."


In [ ]:
def remove_non_ascii(text):
    """
        Remove non-ASCII characters 
    """
    return re.sub(r'[^\x00-\x7f]',r'', text)

In [ ]:
# supprimer les codes assci
train_df["message_clean"] = train_df["message_clean"].apply(lambda x: remove_non_ascii(x))
display(train_df.head())

,Category,Message,message_clean
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif you oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,you dun say so early hor... you c already then...
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i do not think he goes to usf, he lives ar..."


In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

train_df["message_clean"] = train_df['message_clean'].apply(lambda x: remove_punct(x))
train_df.head(10)

,Category,Message,message_clean
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif you oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in a wkly comp to win fa cup final...
3,ham,U dun say so early hor... U c already then say...,you dun say so early hor you c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i do not think he goes to usf he lives aro...
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey there darling it is been weeks no...
6,ham,Even my brother is not like to speak with me. ...,even my brother is not like to speak with me t...
7,ham,As per your request 'Melle Melle (Oru Minnamin...,as per your request melle melle oru minnaminun...
8,spam,WINNER!! As a valued network customer you have...,winner as a valued network customer you have b...
9,spam,Had your mobile 11 months or more? U R entitle...,had your mobile months or more you r entitled...


In [ ]:
!pip install NLTK

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# l'extraction des mots 

from nltk.tokenize import word_tokenize

train_df['tokenized'] = train_df['message_clean'].apply(word_tokenize)
train_df.head()

,Category,Message,message_clean,tokenized
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...,"[go, until, jurong, point, crazy, available, o..."
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif you oni,"[ok, lar, joking, wif, you, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in a wkly comp to win fa cup final...,"[free, entry, in, a, wkly, comp, to, win, fa, ..."
3,ham,U dun say so early hor... U c already then say...,you dun say so early hor you c already then say,"[you, dun, say, so, early, hor, you, c, alread..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i do not think he goes to usf he lives aro...,"[nah, i, do, not, think, he, goes, to, usf, he..."


In [ ]:
# supprimer les mots d'arrete

nltk.download("stopwords")
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))
train_df['stopwords_removed'] = train_df['tokenized'].apply(lambda x: [word for word in x if word not in stop])
train_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Category,Message,message_clean,tokenized,stopwords_removed
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif you oni,"[ok, lar, joking, wif, you, oni]","[ok, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in a wkly comp to win fa cup final...,"[free, entry, in, a, wkly, comp, to, win, fa, ...","[free, entry, wkly, comp, win, fa, cup, final,..."
3,ham,U dun say so early hor... U c already then say...,you dun say so early hor you c already then say,"[you, dun, say, so, early, hor, you, c, alread...","[dun, say, early, hor, c, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i do not think he goes to usf he lives aro...,"[nah, i, do, not, think, he, goes, to, usf, he...","[nah, think, goes, usf, lives, around, though]"


In [ ]:
from nltk.stem import PorterStemmer

def porter_stemmer(text):
    """
        Stem words in list of tokenized words with PorterStemmer
    """
    stemmer = nltk.PorterStemmer()
    stems = [stemmer.stem(i) for i in text]
    return stems

In [ ]:
%time 
# stemming
train_df['porter_stemmer'] = train_df['stopwords_removed'].apply(lambda x: porter_stemmer(x))
print(train_df['porter_stemmer'][1])
print(train_df['stopwords_removed'][1])

#train_df.head()

CPU times: user 0 ns, sys: 4 µs, total: 4 µs
Wall time: 8.11 µs
['ok', 'lar', 'joke', 'wif', 'oni']
['ok', 'lar', 'joking', 'wif', 'oni']


In [ ]:
from nltk.stem import SnowballStemmer

def snowball_stemmer(text):
    """
        Stem words in list of tokenized words with SnowballStemmer
    """
    stemmer = nltk.SnowballStemmer("english")
    stems = [stemmer.stem(i) for i in text]
    return stems

In [ ]:
%time 

train_df['snowball_stemmer'] = train_df['stopwords_removed'].apply(lambda x: snowball_stemmer(x))
print(train_df['snowball_stemmer'][3])
print(train_df['stopwords_removed'][3])
print(train_df['snowball_stemmer'][76])
print(train_df['stopwords_removed'][76])
print(train_df['snowball_stemmer'][32])
print(train_df['stopwords_removed'][32])
#train_df.head()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs
['dun', 'say', 'earli', 'hor', 'c', 'alreadi', 'say']
['dun', 'say', 'early', 'hor', 'c', 'already', 'say']
['cool', 'gentleman', 'treat', 'digniti', 'respect']
['cool', 'gentleman', 'treat', 'dignity', 'respect']
['k', 'tell', 'anyth']
['k', 'tell', 'anything']


In [ ]:
from nltk.stem import LancasterStemmer

def lancaster_stemmer(text):
    """
        Stem words in list of tokenized words with LancasterStemmer
    """
    stemmer = nltk.LancasterStemmer()
    stems = [stemmer.stem(i) for i in text]
    return stems

In [ ]:
%time 

train_df['lancaster_stemmer'] = train_df['stopwords_removed'].apply(lambda x: lancaster_stemmer(x))
print(train_df['lancaster_stemmer'][3])
print(train_df['stopwords_removed'][3])
print(train_df['lancaster_stemmer'][76])
print(train_df['stopwords_removed'][76])
print(train_df['lancaster_stemmer'][32])
print(train_df['stopwords_removed'][32])
#train_df.head()

CPU times: user 13 µs, sys: 1 µs, total: 14 µs
Wall time: 20 µs
['dun', 'say', 'ear', 'hor', 'c', 'already', 'say']
['dun', 'say', 'early', 'hor', 'c', 'already', 'say']
['cool', 'gentlem', 'tre', 'dign', 'respect']
['cool', 'gentleman', 'treat', 'dignity', 'respect']
['k', 'tel', 'anyth']
['k', 'tell', 'anything']


In [ ]:
nltk.download('wordnet')
nltk.download('brown')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
from nltk.corpus import wordnet
from nltk.corpus import brown

wordnet_map = {"N":wordnet.NOUN, 
               "V":wordnet.VERB, 
               "J":wordnet.ADJ, 
               "R":wordnet.ADV
              }
    
train_sents = brown.tagged_sents(categories='news')
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)

def pos_tag_wordnet(text, pos_tag_type="pos_tag"):
    """
        Create pos_tag with wordnet format
    """
    pos_tagged_text = t2.tag(text)
    
    # map the pos tagging output with wordnet output 
    pos_tagged_text = [(word, wordnet_map.get(pos_tag[0])) if pos_tag[0] in wordnet_map.keys() else (word, wordnet.NOUN) for (word, pos_tag) in pos_tagged_text ]
    return pos_tagged_text

In [ ]:
pos_tag_wordnet(train_df['stopwords_removed'][123])


[('todays', 'n'),
 ('voda', 'n'),
 ('numbers', 'n'),
 ('ending', 'v'),
 ('selected', 'v'),
 ('receive', 'v'),
 ('award', 'n'),
 ('match', 'v'),
 ('please', 'v'),
 ('call', 'v'),
 ('quoting', 'n'),
 ('claim', 'n'),
 ('code', 'n'),
 ('standard', 'a'),
 ('rates', 'n'),
 ('app', 'n')]

In [ ]:
%time 

train_df['combined_postag_wnet'] = train_df['stopwords_removed'].apply(lambda x: pos_tag_wordnet(x))

train_df.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


,Category,Message,message_clean,tokenized,stopwords_removed,porter_stemmer,snowball_stemmer,lancaster_stemmer,combined_postag_wnet
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazi, avail, bugi, n, gre...","[go, jurong, point, crazi, avail, bugi, n, gre...","[go, jurong, point, crazy, avail, bug, n, gre,...","[(go, v), (jurong, n), (point, n), (crazy, a),..."
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif you oni,"[ok, lar, joking, wif, you, oni]","[ok, lar, joking, wif, oni]","[ok, lar, joke, wif, oni]","[ok, lar, joke, wif, oni]","[ok, lar, jok, wif, on]","[(ok, n), (lar, n), (joking, n), (wif, n), (on..."
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in a wkly comp to win fa cup final...,"[free, entry, in, a, wkly, comp, to, win, fa, ...","[free, entry, wkly, comp, win, fa, cup, final,...","[free, entri, wkli, comp, win, fa, cup, final,...","[free, entri, wkli, comp, win, fa, cup, final,...","[fre, entry, wkly, comp, win, fa, cup, fin, tk...","[(free, a), (entry, n), (wkly, n), (comp, n), ..."
3,ham,U dun say so early hor... U c already then say...,you dun say so early hor you c already then say,"[you, dun, say, so, early, hor, you, c, alread...","[dun, say, early, hor, c, already, say]","[dun, say, earli, hor, c, alreadi, say]","[dun, say, earli, hor, c, alreadi, say]","[dun, say, ear, hor, c, already, say]","[(dun, n), (say, v), (early, r), (hor, n), (c,..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i do not think he goes to usf he lives aro...,"[nah, i, do, not, think, he, goes, to, usf, he...","[nah, think, goes, usf, lives, around, though]","[nah, think, goe, usf, live, around, though]","[nah, think, goe, usf, live, around, though]","[nah, think, goe, usf, liv, around, though]","[(nah, n), (think, v), (goes, v), (usf, n), (l..."


In [ ]:
from nltk.stem import WordNetLemmatizer

def lemmatize_word(text):
    """
        Lemmatize the tokenized words
    """

    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemma

In [ ]:
%time 

# Test without POS Tagging
lemmatizer = WordNetLemmatizer()

train_df['lemmatize_word_wo_pos'] = train_df['stopwords_removed'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
train_df['lemmatize_word_wo_pos'] = train_df['lemmatize_word_wo_pos'].apply(lambda x: [word for word in x if word not in stop])
#train_df.head()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs


In [ ]:
print(train_df["combined_postag_wnet"][8])
print(train_df["lemmatize_word_wo_pos"][8])

[('winner', 'n'), ('valued', 'v'), ('network', 'n'), ('customer', 'n'), ('selected', 'v'), ('receivea', 'n'), ('prize', 'n'), ('reward', 'n'), ('claim', 'n'), ('call', 'v'), ('claim', 'n'), ('code', 'n'), ('kl', 'n'), ('valid', 'a'), ('hours', 'n')]
['winner', 'valued', 'network', 'customer', 'selected', 'receivea', 'prize', 'reward', 'claim', 'call', 'claim', 'code', 'kl', 'valid', 'hour']


In [ ]:
%time 

# Test with POS Tagging
lemmatizer = WordNetLemmatizer()

train_df['lemmatize_word_w_pos'] = train_df['combined_postag_wnet'].apply(lambda x: lemmatize_word(x))
train_df['lemmatize_word_w_pos'] = train_df['lemmatize_word_w_pos'].apply(lambda x: [word for word in x if word not in stop]) # double check to remove stop words
train_df['lemmatize_text'] = [' '.join(map(str, l)) for l in train_df['lemmatize_word_w_pos']] # join back to text

train_df.head()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.54 µs


,Category,Message,message_clean,tokenized,stopwords_removed,porter_stemmer,snowball_stemmer,lancaster_stemmer,combined_postag_wnet,lemmatize_word_wo_pos,lemmatize_word_w_pos,lemmatize_text
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazi, avail, bugi, n, gre...","[go, jurong, point, crazi, avail, bugi, n, gre...","[go, jurong, point, crazy, avail, bug, n, gre,...","[(go, v), (jurong, n), (point, n), (crazy, a),...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazy, available, bugis, n...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif you oni,"[ok, lar, joking, wif, you, oni]","[ok, lar, joking, wif, oni]","[ok, lar, joke, wif, oni]","[ok, lar, joke, wif, oni]","[ok, lar, jok, wif, on]","[(ok, n), (lar, n), (joking, n), (wif, n), (on...","[ok, lar, joking, wif, oni]","[ok, lar, joking, wif, oni]",ok lar joking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in a wkly comp to win fa cup final...,"[free, entry, in, a, wkly, comp, to, win, fa, ...","[free, entry, wkly, comp, win, fa, cup, final,...","[free, entri, wkli, comp, win, fa, cup, final,...","[free, entri, wkli, comp, win, fa, cup, final,...","[fre, entry, wkly, comp, win, fa, cup, fin, tk...","[(free, a), (entry, n), (wkly, n), (comp, n), ...","[free, entry, wkly, comp, win, fa, cup, final,...","[free, entry, wkly, comp, win, fa, cup, final,...",free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,you dun say so early hor you c already then say,"[you, dun, say, so, early, hor, you, c, alread...","[dun, say, early, hor, c, already, say]","[dun, say, earli, hor, c, alreadi, say]","[dun, say, earli, hor, c, alreadi, say]","[dun, say, ear, hor, c, already, say]","[(dun, n), (say, v), (early, r), (hor, n), (c,...","[dun, say, early, hor, c, already, say]","[dun, say, early, hor, c, already, say]",dun say early hor c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i do not think he goes to usf he lives aro...,"[nah, i, do, not, think, he, goes, to, usf, he...","[nah, think, goes, usf, lives, around, though]","[nah, think, goe, usf, live, around, though]","[nah, think, goe, usf, live, around, though]","[nah, think, goe, usf, liv, around, though]","[(nah, n), (think, v), (goes, v), (usf, n), (l...","[nah, think, go, usf, life, around, though]","[nah, think, go, usf, live, around, though]",nah think go usf live around though


In [ ]:
print(train_df["Message"][8])
print(train_df["combined_postag_wnet"][8])
print(train_df["lemmatize_word_wo_pos"][8])
print(train_df["lemmatize_word_w_pos"][8])

WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
[('winner', 'n'), ('valued', 'v'), ('network', 'n'), ('customer', 'n'), ('selected', 'v'), ('receivea', 'n'), ('prize', 'n'), ('reward', 'n'), ('claim', 'n'), ('call', 'v'), ('claim', 'n'), ('code', 'n'), ('kl', 'n'), ('valid', 'a'), ('hours', 'n')]
['winner', 'valued', 'network', 'customer', 'selected', 'receivea', 'prize', 'reward', 'claim', 'call', 'claim', 'code', 'kl', 'valid', 'hour']
['winner', 'value', 'network', 'customer', 'select', 'receivea', 'prize', 'reward', 'claim', 'call', 'claim', 'code', 'kl', 'valid', 'hour']


In [ ]:
display(train_df["Message"][0], train_df["lemmatize_text"][0])
display(train_df["Message"][5], train_df["lemmatize_text"][5])
display(train_df["Message"][10], train_df["lemmatize_text"][10])
display(train_df["Message"][15], train_df["lemmatize_text"][15])
display(train_df["Message"][20], train_df["lemmatize_text"][20])

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

'go jurong point crazy available bugis n great world la e buffet cine get amore wat'

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"

'freemsg hey darling week word back would like fun still tb ok xxx std chgs send rcv'

"I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today."

'go home soon want talk stuff anymore tonight k cried enough today'

'XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL'

'xxxmobilemovieclub use credit click wap link next txt message click xxxmobilemovieclubcomnqjkgighjjgcbl'

'Is that seriously how you spell his name?'

'seriously spell name'

In [ ]:
# Install the main polygot and other neccesary packages
!pip install pyicu
!pip install pycld2
!pip install polyglot

     |████████████████████████████████| 305 kB 12.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyicu: filename=PyICU-2.9-cp37-cp37m-linux_x86_64.whl size=1375714 sha256=0f3e5ee2ab25abcafcce5470ccc0e13c88df0d438e815221aecff06c501113be
  Stored in directory: /root/.cache/pip/wheels/28/88/93/6c1b06361e4cbd4e7f793fb456729f69798f9aa3fc2a791cd7
Successfully built pyicu
     |████████████████████████████████| 41.4 MB 1.3 MB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834322 sha256=1b682b971c6064afc14e7914b66e2eab2e39b55ca9b2b5245a2f1755848bbc74
  Stored in directory: /root/.cache/pip/wheels/ed/e4/58/ed2e9f43c07d617cc81fe7aff0fc6e42b16c9cf6afe960b614
Successfully built pycld2
     |████████████████████████████████| 126 kB 11.9 MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52577 sha256=d5b353793f4c5a2b47d6

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def cv(data, ngram = 1, MAX_NB_WORDS = 75000):
    count_vectorizer = CountVectorizer(ngram_range = (ngram, ngram), max_features = MAX_NB_WORDS)
    emb = count_vectorizer.fit_transform(data).toarray()
    print("count vectorize with", str(np.array(emb).shape[1]), "features")
    return emb, count_vectorizer

In [ ]:
def print_out(emb, feat, ngram, compared_sentence=0):
    print(ngram,"bag-of-words: ")
    print(feat.get_feature_names(), "\n")
    print(ngram,"bag-of-feature: ")
    print(test_cv_1gram.vocabulary_, "\n")
    print("BoW matrix:")
    print(pd.DataFrame(emb.transpose(), index = feat.get_feature_names()).head(), "\n")
    print(ngram,"vector example:")
    print(train_df["lemmatize_text"][compared_sentence])
    print(emb[compared_sentence], "\n")

In [ ]:
test_corpus = train_df["lemmatize_text"][:5].tolist()
print("The test corpus: ", test_corpus, "\n")

test_cv_em_1gram, test_cv_1gram = cv(test_corpus, ngram=1)
print_out(test_cv_em_1gram, test_cv_1gram, ngram="Uni-gram")

The test corpus:  ['go jurong point crazy available bugis n great world la e buffet cine get amore wat', 'ok lar joking wif oni', 'free entry wkly comp win fa cup final tkts st may text fa receive entry questionstd txt ratetcs apply', 'dun say early hor c already say', 'nah think go usf live around though'] 

count vectorize with 47 features
Uni-gram bag-of-words: 
['already', 'amore', 'apply', 'around', 'available', 'buffet', 'bugis', 'cine', 'comp', 'crazy', 'cup', 'dun', 'early', 'entry', 'fa', 'final', 'free', 'get', 'go', 'great', 'hor', 'joking', 'jurong', 'la', 'lar', 'live', 'may', 'nah', 'ok', 'oni', 'point', 'questionstd', 'ratetcs', 'receive', 'say', 'st', 'text', 'think', 'though', 'tkts', 'txt', 'usf', 'wat', 'wif', 'win', 'wkly', 'world'] 

Uni-gram bag-of-feature: 
{'go': 18, 'jurong': 22, 'point': 30, 'crazy': 9, 'available': 4, 'bugis': 6, 'great': 19, 'world': 46, 'la': 23, 'buffet': 5, 'cine': 7, 'get': 17, 'amore': 1, 'wat': 42, 'ok': 28, 'lar': 24, 'joking': 21, 'w

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
test_cv_em_2gram, test_cv_2gram = cv(test_corpus, ngram=2)
print_out(test_cv_em_2gram, test_cv_2gram, ngram="Bi-gram")

count vectorize with 46 features
Bi-gram bag-of-words: 
['already say', 'amore wat', 'around though', 'available bugis', 'buffet cine', 'bugis great', 'cine get', 'comp win', 'crazy available', 'cup final', 'dun say', 'early hor', 'entry questionstd', 'entry wkly', 'fa cup', 'fa receive', 'final tkts', 'free entry', 'get amore', 'go jurong', 'go usf', 'great world', 'hor already', 'joking wif', 'jurong point', 'la buffet', 'lar joking', 'live around', 'may text', 'nah think', 'ok lar', 'point crazy', 'questionstd txt', 'ratetcs apply', 'receive entry', 'say early', 'st may', 'text fa', 'think go', 'tkts st', 'txt ratetcs', 'usf live', 'wif oni', 'win fa', 'wkly comp', 'world la'] 

Bi-gram bag-of-feature: 
{'go': 18, 'jurong': 22, 'point': 30, 'crazy': 9, 'available': 4, 'bugis': 6, 'great': 19, 'world': 46, 'la': 23, 'buffet': 5, 'cine': 7, 'get': 17, 'amore': 1, 'wat': 42, 'ok': 28, 'lar': 24, 'joking': 21, 'wif': 43, 'oni': 29, 'free': 16, 'entry': 13, 'wkly': 45, 'comp': 8, 'win': 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
test_cv_em_3gram, test_cv_3gram = cv(test_corpus, ngram=3)
print_out(test_cv_em_2gram, test_cv_2gram, ngram="Tri-gram")

count vectorize with 41 features
Tri-gram bag-of-words: 
['already say', 'amore wat', 'around though', 'available bugis', 'buffet cine', 'bugis great', 'cine get', 'comp win', 'crazy available', 'cup final', 'dun say', 'early hor', 'entry questionstd', 'entry wkly', 'fa cup', 'fa receive', 'final tkts', 'free entry', 'get amore', 'go jurong', 'go usf', 'great world', 'hor already', 'joking wif', 'jurong point', 'la buffet', 'lar joking', 'live around', 'may text', 'nah think', 'ok lar', 'point crazy', 'questionstd txt', 'ratetcs apply', 'receive entry', 'say early', 'st may', 'text fa', 'think go', 'tkts st', 'txt ratetcs', 'usf live', 'wif oni', 'win fa', 'wkly comp', 'world la'] 

Tri-gram bag-of-feature: 
{'go': 18, 'jurong': 22, 'point': 30, 'crazy': 9, 'available': 4, 'bugis': 6, 'great': 19, 'world': 46, 'la': 23, 'buffet': 5, 'cine': 7, 'get': 17, 'amore': 1, 'wat': 42, 'ok': 28, 'lar': 24, 'joking': 21, 'wif': 43, 'oni': 29, 'free': 16, 'entry': 13, 'wkly': 45, 'comp': 8, 'win'

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(train_df["lemmatize_text"][123])
print(train_df["lemmatize_text"][1223])
print(train_df["lemmatize_text"][2123])
print(train_df["lemmatize_text"][4123])

today voda number end select receive award match please call quoting claim code standard rate app
teach apps da come college
know result
doc prescribed morphine pain med enough wait mom bring med kick fast go try later


In [ ]:
%time 

# implement into the whole dataset
train_df_corpus = train_df["lemmatize_text"].tolist()

train_df_em_1gram, vc_1gram = cv(train_df_corpus, 1)
train_df_em_2gram, vc_2gram = cv(train_df_corpus, 2)
train_df_em_3gram, vc_3gram = cv(train_df_corpus, 3)

#print_out(train_df_em_1gram, vc_1gram, ngram="Uni-gram")
#print_out(train_df_em_2gram, vc_2gram, ngram="Bi-gram")
#print_out(train_df_em_3gram, vc_3gram, ngram="Tri-gram")

print(len(train_df_corpus))
print(train_df_em_1gram.shape)
print(train_df_em_2gram.shape)
print(train_df_em_3gram.shape)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.68 µs
count vectorize with 7437 features
count vectorize with 28343 features
count vectorize with 28171 features
5572
(5572, 7437)
(5572, 28343)
(5572, 28171)


In [ ]:
del train_df_em_1gram, train_df_em_2gram, train_df_em_3gram



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def TFIDF(data, ngram = 1, MAX_NB_WORDS = 75000):
    tfidf_x = TfidfVectorizer(ngram_range = (ngram, ngram), max_features = MAX_NB_WORDS)
    emb = tfidf_x.fit_transform(data).toarray()
    print("tf-idf with", str(np.array(emb).shape[1]), "features")
    return emb, tfidf_x

In [ ]:
test_corpus = train_df["lemmatize_text"][:5].tolist()
print("The test corpus: ", test_corpus, "\n")

test_tfidf_em_1gram, test_tfidf_1gram = TFIDF(test_corpus, ngram=1)
print_out(test_tfidf_em_1gram, test_tfidf_1gram, ngram="Uni-gram")

The test corpus:  ['go jurong point crazy available bugis n great world la e buffet cine get amore wat', 'ok lar joking wif oni', 'free entry wkly comp win fa cup final tkts st may text fa receive entry questionstd txt ratetcs apply', 'dun say early hor c already say', 'nah think go usf live around though'] 

tf-idf with 47 features
Uni-gram bag-of-words: 
['already', 'amore', 'apply', 'around', 'available', 'buffet', 'bugis', 'cine', 'comp', 'crazy', 'cup', 'dun', 'early', 'entry', 'fa', 'final', 'free', 'get', 'go', 'great', 'hor', 'joking', 'jurong', 'la', 'lar', 'live', 'may', 'nah', 'ok', 'oni', 'point', 'questionstd', 'ratetcs', 'receive', 'say', 'st', 'text', 'think', 'though', 'tkts', 'txt', 'usf', 'wat', 'wif', 'win', 'wkly', 'world'] 

Uni-gram bag-of-feature: 
{'go': 18, 'jurong': 22, 'point': 30, 'crazy': 9, 'available': 4, 'bugis': 6, 'great': 19, 'world': 46, 'la': 23, 'buffet': 5, 'cine': 7, 'get': 17, 'amore': 1, 'wat': 42, 'ok': 28, 'lar': 24, 'joking': 21, 'wif': 43, 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
test_tfidf_em_2gram, test_tfidf_2gram = TFIDF(test_corpus, ngram=2)
print_out(test_tfidf_em_2gram, test_tfidf_2gram, ngram="Bi-gram")

tf-idf with 46 features
Bi-gram bag-of-words: 
['already say', 'amore wat', 'around though', 'available bugis', 'buffet cine', 'bugis great', 'cine get', 'comp win', 'crazy available', 'cup final', 'dun say', 'early hor', 'entry questionstd', 'entry wkly', 'fa cup', 'fa receive', 'final tkts', 'free entry', 'get amore', 'go jurong', 'go usf', 'great world', 'hor already', 'joking wif', 'jurong point', 'la buffet', 'lar joking', 'live around', 'may text', 'nah think', 'ok lar', 'point crazy', 'questionstd txt', 'ratetcs apply', 'receive entry', 'say early', 'st may', 'text fa', 'think go', 'tkts st', 'txt ratetcs', 'usf live', 'wif oni', 'win fa', 'wkly comp', 'world la'] 

Bi-gram bag-of-feature: 
{'go': 18, 'jurong': 22, 'point': 30, 'crazy': 9, 'available': 4, 'bugis': 6, 'great': 19, 'world': 46, 'la': 23, 'buffet': 5, 'cine': 7, 'get': 17, 'amore': 1, 'wat': 42, 'ok': 28, 'lar': 24, 'joking': 21, 'wif': 43, 'oni': 29, 'free': 16, 'entry': 13, 'wkly': 45, 'comp': 8, 'win': 44, 'fa':

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
test_tfidf_em_3gram, test_tfidf_3gram = TFIDF(test_corpus, ngram=3)
print_out(test_tfidf_em_3gram, test_tfidf_3gram, ngram="Tri-gram")

tf-idf with 41 features
Tri-gram bag-of-words: 
['available bugis great', 'buffet cine get', 'bugis great world', 'cine get amore', 'comp win fa', 'crazy available bugis', 'cup final tkts', 'dun say early', 'early hor already', 'entry questionstd txt', 'entry wkly comp', 'fa cup final', 'fa receive entry', 'final tkts st', 'free entry wkly', 'get amore wat', 'go jurong point', 'go usf live', 'great world la', 'hor already say', 'joking wif oni', 'jurong point crazy', 'la buffet cine', 'lar joking wif', 'live around though', 'may text fa', 'nah think go', 'ok lar joking', 'point crazy available', 'questionstd txt ratetcs', 'receive entry questionstd', 'say early hor', 'st may text', 'text fa receive', 'think go usf', 'tkts st may', 'txt ratetcs apply', 'usf live around', 'win fa cup', 'wkly comp win', 'world la buffet'] 

Tri-gram bag-of-feature: 
{'go': 18, 'jurong': 22, 'point': 30, 'crazy': 9, 'available': 4, 'bugis': 6, 'great': 19, 'world': 46, 'la': 23, 'buffet': 5, 'cine': 7, 'ge

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
%time 

# implement into the whole dataset
train_df_corpus = train_df["lemmatize_text"].tolist()
train_df_tfidf_1gram, tfidf_1gram = TFIDF(train_df_corpus, 1)
train_df_tfidf_2gram, tfidf_2gram = TFIDF(train_df_corpus, 2)
train_df_tfidf_3gram, tfidf_3gram = TFIDF(train_df_corpus, 3)

print(len(train_df_corpus))
print(train_df_tfidf_1gram.shape)
print(train_df_tfidf_1gram.shape)
print(train_df_tfidf_1gram.shape)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 21.2 µs
tf-idf with 7437 features
tf-idf with 28343 features
tf-idf with 28171 features
5572
(5572, 7437)
(5572, 7437)
(5572, 7437)


In [ ]:
del train_df_tfidf_1gram, train_df_tfidf_2gram, train_df_tfidf_3gram

